In [3]:
import re
import os

In [10]:
# Defining Helper variables to be used later

labelList = label_l = ["O", "B-positive", "B-negative", "B-neutral", "I-positive", "I-negative", "I-neutral"]
d = {   
        "START": 0,
        "O": 0,
        "B-positive": 0,
        "B-negative": 0,
        "B-neutral": 0,
        "I-positive": 0,
        "I-negative": 0,
        "I-neutral": 0,
        "END": 0
    }

folderPath = os.path.abspath(os.getcwd())

In [23]:
# Helper functions to read and parse data
def readFile(filePath: str):
    with open(filePath, "r", encoding="utf-8") as f:
        return f.readlines()
    
def processFile(file: list):
    return [word[:len(word)-1] for word in file]

### Part 1

1. Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation):
<br>
$$
e(x|y) = \frac{{\text{{Count}}(y \rightarrow x)}}{{\text{{Count}}(y)}}
$$



In [17]:
# Calculating Emissions Function

def calcCountofEachWord(file: list):
    for i in range(len(file)):
        if file[i] != "":
            l = file[i].split()
            entity = l[0]
            label = l[1]
            key = f"{entity}_{label}"
            if key in d:
                d[key] += 1
                d[label] += 1
            else:
                d[key] = 1
                d[label] += 1
    return d

def calcEmission(input_d: dict):
    emissionResult = 1
    output_d = {}
    for key, value in input_d.items():
        if key not in label_l:
            label = key.split("_")[1]
            val = value / input_d[label]
            output_d[key] = val
            emissionResult *= val
    return emissionResult, output_d

In [30]:
# Calling the functions

# Defining the filePath for the training dataset
EsTrainFilePath = "../Data/ES/train"
filePath = os.path.join(folderPath, EsTrainFilePath)

# Processing the file to separate line by line
processedFile = processFile(readFile(filePath=filePath))

# Calculating the count of each word to the label
countDict = calcCountofEachWord(processedFile)
# print(countDict)

# Calculating the emission value for each unique word
emissionVal, emissionsDict = calcEmission(countDict)
print('Emissions Dict: \n', emissionsDict)

{'Estuvimos_O': 0.0002066471499913897, 'hace_O': 0.0008954709832960221, 'poco_O': 0.0018942655415877389, 'mi_O': 0.0024797657998966763, 'pareja_O': 0.00044773549164801103, 'y_O': 0.035267780265197175, 'yo_O': 0.0012398828999483382, 'comiendo_O': 0.00034441191665231615, 'resultó_O': 0.00013776476666092647, 'todo_O': 0.003960737041501636, 'muy_O': 0.01363871189943172, 'bien_O': 0.005682796624763217, ',_O': 0.05731014293094541, 'tanto_O': 0.001343206474944033, 'la_O': 0.02600309970724987, 'comida_B-positive': 0.1456896551724138, 'el_O': 0.022111245049078698, 'vino_B-positive': 0.005172413793103448, 'trato_B-positive': 0.03793103448275862, 'decoración_B-positive': 0.00603448275862069, '…_O': 0.0015498536249354228, 'nos_O': 0.005028413983123816, 'gustó_O': 0.00037885310831754776, 'mucho_O': 0.0018253831582572756, '._O': 0.055898054072670914, 'Por_O': 0.0010332357499569485, 'poner_O': 0.00017220595832615808, 'algún_O': 0.0002066471499913897, 'pero_O': 0.006578267608059239, 'quizá_O': 0.00013

2. Set k to 1, implement this fix into your function for computing the emission parameters

$$
e(x|y) = \begin{cases}
\frac{{\text{{Count}}(y \rightarrow x)}}{{\text{{Count}}(y)+k}}, & \text{{if the word token }} x \text{{ appears in the training set}} \\
\frac{k}{{\text{{Count}}(y)+k}}, & \text{{if word token }} x \text{{ is the special token \#UNK\#}}
\end{cases}
